In [2]:
import numpy as np
import pandas as pd

file_path = r"../data/hotel_bookings.csv"
data = pd.read_csv(file_path)
#  查看数据的行、列
print(data.shape)

(119390, 32)


In [11]:
# 查看数据非空数据量
data.count()

# NAN统计,county、agent、company、children中这四列出现nan数据
data.isna().sum() 

# # 数据整体描述
# data.describe()

# # 数据信息
# data.info()

# # 简单查看一下数据
# data.head()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

通过describe()方法看出：
children 这一列少了4个值，且均值，4分位值均为0，因此填充为0值
country、agent、company：缺失值较多，分别为488，16340，112593。因此考虑去掉这些列，或者单独拿出来进行分析